In [2]:
import os
from os import listdir
import time
import gzip
import json

# Torch libs
import torch
import torch.nn as nn
import torch.nn.functional as F

from torch.utils.data import Dataset, DataLoader

from torch.utils.data.sampler import WeightedRandomSampler

import torchvision.transforms as T
from torchvision.utils import make_grid
from torchvision.models import resnet50
import torchvision.transforms as transforms
from torchvision.io import read_image
import torch.optim as optim

from sklearn.model_selection import train_test_split

# Data libs
import pandas as pd
import numpy as np
# import cv2
from PIL import Image, ImageDraw
import matplotlib.pyplot as plt
%matplotlib inline

In [3]:
import pandas as pd
import gzip

def parse(path):
  g = gzip.open(path, 'rb')
  for l in g:
    yield json.loads(l)

def getDF(path):
  i = 0
  df = {}
  for d in parse(path):
    df[i] = d
    i += 1
  return pd.DataFrame.from_dict(df, orient='index')


In [4]:
os.chdir('data')

In [5]:
df = getDF('Luxury_Beauty_5.json.gz')

In [6]:
display(df.head(10))

,overall,verified,reviewTime,reviewerID,asin,style,reviewerName,reviewText,summary,unixReviewTime,vote,image
0,5.0,True,"01 5, 2018",A2HOI48JK8838M,B00004U9V2,{'Size:': ' 0.9 oz.'},DB,This handcream has a beautiful fragrance. It d...,Beautiful Fragrance,1515110400,NaN,NaN
1,5.0,True,"04 5, 2017",A1YIPEY7HX73S7,B00004U9V2,{'Size:': ' 3.5 oz.'},Ajaey,"wonderful hand lotion, for seriously dry skin,...",wonderful hand lotion,1491350400,NaN,NaN
2,5.0,True,"03 27, 2017",A2QCGHIJ2TCLVP,B00004U9V2,{'Size:': ' 250 g'},D. Jones,"Best hand cream around. Silky, thick, soaks i...",Best hand cream around,1490572800,NaN,NaN
3,5.0,True,"03 20, 2017",A2R4UNHFJBA6PY,B00004U9V2,{'Size:': ' 3.5 oz.'},Amazon Customer,Thanks!!,Five Stars,1489968000,NaN,NaN
4,5.0,True,"02 28, 2017",A2QCGHIJ2TCLVP,B00004U9V2,{'Size:': ' 0.9 oz.'},D. Jones,Great hand lotion. Soaks right in and leaves ...,Great hand lotion!,1488240000,NaN,NaN
5,5.0,True,"02 25, 2017",A1606LA683WZZU,B00004U9V2,{'Size:': ' 250 g'},Amr,Great product. Doesn't leave you hands feeling...,Five Stars,1487980800,NaN,NaN
6,5.0,True,"02 25, 2017",A1606LA683WZZU,B00004U9V2,{'Size:': ' 3.5 oz.'},Amr,Great product. Doesn't leave you hands feeling...,Five Stars,1487980800,NaN,NaN
7,5.0,True,"01 30, 2017",A1606LA683WZZU,B00004U9V2,{'Size:': ' 0.9 oz.'},Amr,Just as described. Arrived on time.,Five Stars,1485734400,NaN,NaN
8,4.0,False,"01 24, 2017",A1YY53NQXFKMRN,B00004U9V2,{'Size:': ' 3.5 oz.'},Trixie,Nice lightweight hand cream for the summer.,"Smells good, absorbs quickly",1485216000,NaN,NaN
9,5.0,True,"12 1, 2016",A3R0NQ9E53JHYQ,B00004U9V2,{'Size:': ' 250 g'},T. Hooth,Best hand cream ever.,Five Stars,1480550400,NaN,NaN


In [10]:
print(len(df.index))

34278


In [11]:
df.groupby(by="verified").size()

verified
False    16517
True     17761
dtype: int64

In [14]:
df.groupby(by="summary").size()

summary
 love                                                                                                                    1
" It's very smooth and fills in thinner wrinkles very easily. I was somewhat impressed with the application usability    1
"Better than what I have now"                                                                                            3
"Bliss bronzer" is much better for your face than those damaging sun rays!                                               1
"Euphoria" is the perfect name for this perfume                                                                          1
                                                                                                                        ..
you don't need to pay 100+ dollars worth of soap to get the best shave in your life.                                     1
you have to use a lot if you have thick ...                                                                              2
you need

In [15]:
df.groupby(by="reviewerName").size()

reviewerName
 B Mouk              8
 Catrinka            6
 Daba                1
 Indiangirl          7
 JOKE                4
                    ..
yvonne hankins       8
zenigma283          11
~*~ Sunshine ~*~    16
~Kimber~             8
~purplemoon~         7
Length: 3584, dtype: int64